In [1]:
from scipy.io import loadmat
import os
import scipy.sparse as sp
import numpy as np
import pandas as pd

In [2]:
## CSTR
file_name = "datasets/cstr.mat"
cstr = loadmat(file_name)

## WebKB4
## WebACE
file_name = "datasets/WebACE.mat"
WebACE = loadmat(file_name)

## NG10
## NG20
file_name = "datasets/NG20.mat"
NG20 = loadmat(file_name)

## RCV1
file_name = "datasets/RCV1_ori.mat"
RCV1 = loadmat(file_name)

## Reviews
file_name = "datasets/reviews.mat"
reviews = loadmat(file_name)

## Sports
file_name = "datasets/sports.mat"
sports = loadmat(file_name)

## Classic3
file_name = "datasets/classic3.mat"
classic3 = loadmat(file_name)

## Classic4
file_path = os.path.join('datasets', "classic4.txt")
nb_row = 7095
nb_col = 5896
classic4 = sp.lil_matrix((nb_row, nb_col))


In [ ]:
np.sum(classic4[0:10,1:15].toarray())

In [ ]:
np.unique(WebACE['fea'])

In [ ]:
np.unique(reviews['dtm'].toarray())

In [3]:
from coclust.io.data_loading import load_doc_term_data
from coclust.visualization import (plot_reorganized_matrix,
                                  plot_cluster_top_terms,
                                  plot_max_modularities)
from coclust.evaluation.internal import best_modularity_partition
from coclust.coclustering import CoclustMod
from coclust.coclustering import CoclustInfo

#print(np.isnan(WebACE['fea'].data).any())
#WebACE['fea'][0,0] = float('nan')
#print(np.isnan(WebACE['fea'].data).any())
X = reviews['dtm']#WebACE['fea']

file_name = "NaNs.mat"
NaNs = loadmat(file_name)
NaN = pd.DataFrame(list(zip(NaNs['row'][0], NaNs['col'][0])), columns =['row','col'])

# get the best co-clustering over a range of cluster numbers
model = CoclustInfo(n_row_clusters=5, n_col_clusters=5,
                   missing_indexes=NaN[0:10000])
model.fit(X)

#print(model.modularity)

# plot the reorganized matrix
plot_reorganized_matrix(X, model)

# plot the top terms
#n_terms = 10
#plot_cluster_top_terms(X, labels, n_terms, model)

# plot the modularities over the range of cluster numbers
#plot_max_modularities(modularities, range(2, 6))

Step 2++, Iteration : 0
Step 4++, Iteration : 0
Step 2++, Iteration : 1
Step 4++, Iteration : 1
Step 2++, Iteration : 2
Step 4++, Iteration : 2
Step 2++, Iteration : 3
Step 4++, Iteration : 3
Step 2++, Iteration : 4
Step 4++, Iteration : 4
Step 2++, Iteration : 5
Step 4++, Iteration : 5
Step 2++, Iteration : 6
Step 4++, Iteration : 6
Step 2++, Iteration : 7
Step 4++, Iteration : 7
Step 2++, Iteration : 8
Step 4++, Iteration : 8
Step 2++, Iteration : 9
Step 4++, Iteration : 9
Step 2++, Iteration : 10
Step 4++, Iteration : 10
Step 2++, Iteration : 11
Step 4++, Iteration : 11
Step 2++, Iteration : 12
Step 4++, Iteration : 12
Step 2++, Iteration : 13
Step 4++, Iteration : 13
Step 2++, Iteration : 14
Step 4++, Iteration : 14
Step 2++, Iteration : 15
Step 4++, Iteration : 15
Step 2++, Iteration : 16
Step 4++, Iteration : 16
Step 2++, Iteration : 17
Step 4++, Iteration : 17
Step 2++, Iteration : 18
Step 4++, Iteration : 18


<Figure size 640x480 with 1 Axes>

In [12]:
model.p_kl.todense()*X.sum()

matrix([[ 70090.        ,   4899.        ,  12796.        ,
         150860.        ,  38155.        ],
        [  6865.        , 162488.        ,   2866.        ,
          52030.        ,  91936.        ],
        [  7392.        ,    517.        ,  16530.        ,
          12841.        ,   4562.        ],
        [  7531.        ,  17715.        ,   3830.        ,
          72700.        ,  31951.        ],
        [160191.00000001,   4972.        ,  20717.        ,
         138971.        ,  50633.        ]])

In [8]:
print(model.p_kl.todense())

[[0.00699015 0.0406691  0.00538793 0.01589807 0.05064779]
 [0.00540629 0.0569815  0.00814483 0.12467156 0.03605912]
 [0.05626387 0.03907563 0.01986647 0.00537832 0.18073526]
 [0.00127618 0.00610994 0.02056837 0.0060033  0.00497623]
 [0.13378402 0.03284681 0.01690591 0.00447363 0.12087973]]


In [6]:
print(X.shape)
print(model.Z.shape)
print(model.W.shape)

(4069, 18483)
(4069, 5)
(18483, 5)


In [5]:
print(model.W[0:10,:].todense())
print()
print(model.Z[0:10,:].todense())

[[0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]

[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [4]:
%cd coclust
#!os.chdir('./coclust')
!python3 setup.py install
%cd ..

/Users/amine/Desktop/Coclustering-Imputation/coclust
running install
running bdist_egg
running egg_info
writing coclust.egg-info/PKG-INFO
writing dependency_links to coclust.egg-info/dependency_links.txt
writing entry points to coclust.egg-info/entry_points.txt
writing requirements to coclust.egg-info/requires.txt
writing top-level names to coclust.egg-info/top_level.txt
reading manifest file 'coclust.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'coclust.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.15-x86_64/egg
running install_lib
running build_py
copying coclust/coclustering/coclust_info.py -> build/lib/coclust/coclustering
creating build/bdist.macosx-10.15-x86_64/egg
creating build/bdist.macosx-10.15-x86_64/egg/coclust
creating build/bdist.macosx-10.15-x86_64/egg/coclust/visualization
copying build/lib/coclust/visualization/__init__.py -> build/bdist.macosx-10.15-x86_64/egg/coclust/visualization
copying build/lib/coclu

In [72]:
file_name = "NaNs.mat"
NaNs = loadmat(file_name)

NaNs['row']
NaNs['col']
NaN = pd.DataFrame(list(zip(NaNs['row'][0], NaNs['col'][0])), columns =['row','col'])

In [109]:
NN = np.copy(model.p_kl.todense()).astype(int)
for i in range(NN.shape[0]):
    for j in range(NN.shape[1]):
        NN[i,j] = np.sum(model.Z[:,i])*np.sum(model.W[:,j])
print('Done with NN !')
print('Verification, Real number :',X.shape[0]*X.shape[1],', Found :',np.sum(NN))
for i in NaN.values[0:10000,:]:
    cluster_row = model.Z[i[0],:].todense().tolist()[0].index(1)
    cluster_col = model.W[i[1],:].todense().tolist()[0].index(1)
    print(NN[cluster_row, cluster_col])

Done with NN !
Verification, Real number : 75207327 , Found : 75207327
958470
153058
153058
415337
979274
958470
979274
415337
415337
415337
517128
979274
517128
979274
979274
517128
979274
415337
979274
979274
517128
979274
517128
979274
415337
415337
415337
517128
415337
517128
979274
979274
979274
415337
415337
415337
517128
979274
415337
979274
415337
979274
415337
415337
517128
979274
979274
415337
517128
415337
979274
517128
517128
958470
415337
153058
415337
415337
979274
979274
979274
979274
415337
415337
415337
415337
979274
153058
979274
979274
979274
979274
415337
979274
979274
979274
517128
979274
979274
517128
415337
979274
979274
415337
979274
979274
979274
517128
979274
979274
415337
517128
415337
979274
979274
979274
415337
979274
958470
979274
415337
979274
979274
979274
979274
517128
153058
415337
415337
979274
979274
517128
517128
415337
517128
979274
8789742
4641624
3727971
8603010
8603010
8789742
8789742
3727971
8789742
8789742
8789742
8789742
8789742
8789742
46416

3874920
2046240
2046240
3874920
3874920
1643460
3874920
3874920
1643460
1643460
1643460
3874920
2046240
1643460
3874920
3874920
3874920
3874920
2046240
3874920
3874920
2046240
1643460
3874920
1643460
1643460
1643460
1643460
3874920
3874920
1643460
1643460
2046240
3874920
2046240
8789742
8789742
8789742
8789742
8603010
8789742
4641624
4641624
8789742
8789742
8789742
8789742
4641624
8789742
8789742
8789742
3727971
8603010
4641624
8789742
1373814
3727971
3727971
8789742
3727971
3727971
4641624
8789742
3727971
3727971
8789742
4641624
4641624
8789742
1373814
4641624
8789742
3727971
4641624
8789742
8789742
8789742
4641624
8789742
8789742
8789742
8789742
8789742
8789742
4641624
8789742
8789742
4641624
8789742
8603010
8789742
4641624
8789742
8789742
8789742
8789742
3727971
3727971
8789742
8789742
3727971
3727971
4641624
8789742
8789742
8789742
8789742
8789742
8789742
8789742
3727971
1373814
4641624
8789742
1373814
8603010
8789742
3727971
8789742
4641624
8789742
8789742
8789742
1373814
8789742


3874920
1643460
3874920
3874920
3874920
3792600
1643460
2046240
3874920
3792600
2046240
1643460
3874920
3874920
1643460
3874920
1643460
3874920
605640
3874920
605640
605640
2046240
1643460
1643460
3874920
3874920
3874920
3874920
3874920
3874920
3874920
3874920
2046240
3874920
3874920
1643460
3874920
1643460
3874920
3874920
2046240
3874920
3874920
3874920
1643460
3874920
3874920
2046240
3874920
3874920
3792600
1643460
1643460
1643460
3874920
1643460
3874920
605640
3874920
3874920
1643460
3874920
3874920
3874920
1643460
1643460
605640
3792600
1643460
3874920
3874920
3874920
605640
2046240
3874920
8789742
8789742
3727971
3727971
1373814
4641624
8789742
8789742
8789742
3727971
3727971
8789742
8789742
8789742
4641624
8789742
8789742
8789742
3727971
3727971
8789742
8789742
4641624
4641624
3727971
8789742
8789742
8789742
4641624
8789742
8789742
8789742
8789742
8789742
8789742
8789742
8789742
4641624
8789742
4641624
8789742
3727971
3727971
8789742
8789742
4641624
8789742
3727971
8789742
878974

4641624
8603010
8789742
4641624
4641624
8789742
8789742
8789742
4641624
8789742
3727971
1373814
8789742
4641624
1373814
4641624
8789742
3727971
8789742
8789742
8789742
8603010
8789742
8789742
8789742
8789742
4641624
4641624
8789742
8789742
3727971
8789742
8789742
3727971
3727971
3727971
8789742
4641624
3727971
8789742
8789742
8789742
4641624
8789742
8789742
4641624
8789742
3727971
3727971
3727971
3727971
8789742
8789742
3727971
3727971
4641624
8789742
4641624
8789742
8789742
8789742
8789742
8603010
8789742
4641624
4641624
8789742
8789742
8789742
8789742
4641624
8789742
8789742
8789742
3727971
8603010
4641624
8789742
1373814
3727971
3727971
8789742
3727971
3727971
4641624
8789742
3727971
3727971
8789742
4641624
4641624
8789742
1373814
4641624
8789742
3727971
4641624
8789742
8789742
8789742
4641624
8789742
8789742
8789742
8789742
8789742
8789742
4641624
8789742
8789742
4641624
8789742
8603010
8789742
4641624
8789742
8789742
8789742
8789742
3727971
3727971
8789742
8789742
3727971
3727971


3874920
3874920
3874920
3874920
1643460
3874920
3874920
3874920
3874920
3874920
2046240
3874920
3874920
1643460
2046240
2046240
3874920
1643460
3874920
3874920
3874920
3874920
1643460
3792600
3874920
1643460
3874920
3874920
3874920
3874920
605640
3874920
1643460
3792600
2046240
1643460
2046240
3874920
3874920
3874920
3874920
3874920
1643460
3874920
2046240
1643460
3874920
2046240
1643460
3874920
3874920
3874920
3874920
3874920
1643460
1643460
3792600
3874920
3874920
2046240
2046240
3874920
3874920
3874920
3874920
2046240
3874920
605640
1643460
3874920
1643460
1643460
3792600
3874920
3874920
1643460
605640
3874920
2046240
1643460
3874920
2046240
3874920
605640
3874920
3874920
2046240
3874920
2046240
2046240
3874920
3874920
1643460
8789742
3727971
8789742
8789742
8789742
8603010
8789742
3727971
4641624
4641624
8603010
3727971
8789742
1373814
8603010
8603010
3727971
8789742
8789742
8789742
8789742
3727971
8603010
3727971
4641624
8789742
8789742
8789742
8789742
8789742
8789742
8789742
4641

4596384
8704072
4596384
8519160
3691636
8704072
8704072
8519160
3691636
8704072
8704072
8704072
8704072
3691636
8704072
3691636
4596384
8704072
8704072
8704072
4596384
3691636
3691636
8704072
8704072
1360424
3691636
8704072
8704072
8704072
4596384
3691636
3691636
4596384
4596384
8704072
8704072
8704072
4596384
3691636
8704072
8704072
8704072
8704072
8704072
3691636
8704072
8704072
8704072
8704072
8704072
8704072
8704072
3691636
8704072
3691636
1360424
4596384
8704072
8704072
8704072
8704072
4596384
3691636
4596384
8704072
4596384
8704072
8704072
8704072
8519160
3691636
3691636
4596384
3691636
8704072
8704072
4596384
4596384
8704072
8704072
8704072
8704072
8704072
8704072
8519160
8704072
8704072
3691636
1969830
1062792
314562
853593
2012586
2012586
2012586
2012586
2012586
2012586
1062792
853593
1969830
2012586
2012586
2012586
2012586
853593
2012586
853593
2012586
2012586
1969830
2012586
2012586
2012586
2012586
853593
2012586
853593
1062792
1062792
1062792
1062792
1062792
853593
1062792


4641624
3727971
4641624
8789742
8789742
3727971
3727971
8789742
8789742
8789742
8789742
8789742
3727971
8789742
8789742
8789742
3727971
3727971
4641624
8603010
8789742
8789742
8789742
8789742
8789742
8789742
4641624
3727971
4641624
8789742
4641624
8789742
8789742
3727971
8789742
8789742
4641624
8789742
4641624
8789742
8789742
8603010
3792600
3874920
3874920
2046240
1643460
3792600
2046240
2046240
3874920
1643460
3874920
3874920
3874920
3874920
3874920
1643460
3874920
3874920
3874920
1643460
3874920
3874920
3874920
3874920
1643460
1643460
3792600
2046240
3874920
3874920
1643460
2046240
3874920
2046240
3874920
3792600
1643460
2046240
3874920
3874920
3874920
1643460
2046240
1643460
1643460
3874920
3874920
3874920
3874920
3874920
2046240
3874920
1643460
3874920
3874920
1643460
605640
2046240
3874920
1643460
2046240
2046240
3874920
1643460
605640
3874920
3792600
2046240
2046240
2046240
3792600
3874920
1643460
605640
3874920
2046240
2046240
3874920
3874920
3874920
3874920
3874920
1643460
387

3691636
3691636
8704072
8704072
8704072
8789742
8789742
3727971
8789742
4641624
8789742
8789742
8789742
8789742
8789742
8789742
8789742
4641624
3727971
8789742
8789742
8789742
8789742
8789742
4641624
3727971
3727971
4641624
3727971
8603010
8789742
4641624
8789742
8789742
8789742
8789742
8789742
8789742
8789742
4641624
4641624
3727971
3727971
4641624
4641624
4641624
8603010
3727971
8789742
8789742
8789742
3727971
3727971
4641624
3727971
8789742
8789742
8789742
8789742
8603010
3727971
4641624
8789742
3727971
8789742
4641624
8789742
4641624
8789742
8789742
8789742
1373814
8789742
4641624
4641624
8789742
8789742
8789742
8789742
8789742
8789742
8789742
3727971
1373814
8789742
1373814
8789742
8789742
8789742
8789742
4641624
8789742
4641624
8789742
8789742
3727971
3727971
8789742
8789742
8789742
8789742
8789742
3727971
3727971
4641624
8789742
3727971
8789742
8789742
4641624
8789742
3727971
8603010
3727971
8789742
8789742
3727971
3727971
8789742
8789742
1373814
3727971
8704072
8704072
8704072
